In [ ]:
import pandas as pd
df_test = pd.read_csv('/fs/ess/PAS0536/ppgneogi/TrafficEvents/US_Accidents_March23.csv')
zipcode_df_1 = pd.read_csv('US_ZipCodes_30140.csv')
zipcode_df_2 = pd.read_csv('US_ZipCodes_23021_151.csv')

In [ ]:
df_test.shape

In [ ]:
df_test.columns

In [ ]:
# Combine both the zipcode dataframes (demographics) together
combined_zipcode_df = pd.concat([zipcode_df_1, zipcode_df_2], ignore_index=True)
combined_zipcode_df = combined_zipcode_df.drop_duplicates(subset=['zip_code'])
combined_zipcode_df = combined_zipcode_df.rename(columns={'zip_code': 'Zipcode'})
combined_zipcode_df.reset_index(drop=True, inplace=True)

In [ ]:
# Clean 'Zipcode' column in df_test by removing the '-XXXX' part
df_test['Zipcode'] = df_test['Zipcode'].str.split('-').str[0]

In [ ]:
# Ensure both columns are of the same type (string)
combined_zipcode_df['Zipcode'] = combined_zipcode_df['Zipcode'].astype(str)
# Perform the merge
merged_df = df_test.merge(combined_zipcode_df, on='Zipcode', how='left')
    
# Print the shape of the merged DataFrame
print(merged_df.shape)

In [ ]:
# Define the columns to exclude
columns_to_exclude = [
    'ID', 'Source', 'End_Time', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
    'Street', 'County', 'Country', 'Timezone', 'Airport_Code', 'Wind_Chill(F)',
    'Precipitation(in)', 'Weather_Timestamp', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)', 
    'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)', 'Weather_Condition', 
    'Unnamed: 0', 'County', 'state', 'county', 'time_zone', 'area_code', 'latitude', 
    'longitude', 'population_2019', 'population_2020', 'average_household_income_2019($)', 
    'average_household_income_2020($)', 'population_2005', 'population_2006', 
    'population_2007', 'population_2008', 'population_2009', 'population_2010', 
    'population_2011', 'population_2012', 'population_2013', 'population_2014', 
    'population_2015', 'population_2016', 'population_2017', 'population_2018', 
    'owner_occupied_home_values_from_$1_to_$24999_dollars', 
    'owner_occupied_home_values_from_$25000_to_$49999_dollars', 
    'owner_occupied_home_values_from_$50000_to_$99999_dollars', 
    'owner_occupied_home_values_from_$100000_to_$149999_dollars', 
    'owner_occupied_home_values_from_$150000_to_$199999_dollars', 
    'owner_occupied_home_values_from_$200000_to_$399999_dollars', 
    'owner_occupied_home_values_from_$400000_to_$749999_dollars', 
    'owner_occupied_home_values_more_than_$750000_dollars', 
    'rented_housing_number_of_rooms_studio', 
    'rented_housing_number_of_rooms_1_bedroom', 
    'rented_housing_number_of_rooms_2_bedroom', 
    'rented_housing_number_of_rooms_3_or_more_bedroom', 
    'average_household_income_2005($)', 'average_household_income_2006($)', 
    'average_household_income_2007($)', 'average_household_income_2008($)', 
    'average_household_income_2009($)', 'average_household_income_2010($)', 
    'average_household_income_2011($)', 'average_household_income_2012($)', 
    'average_household_income_2013($)', 'average_household_income_2014($)', 
    'average_household_income_2015($)', 'average_household_income_2016($)', 
    'average_household_income_2017($)', 'average_household_income_2018($)', 
    'household_income_less_than_25000_dollars', 
    'household_income_from_45000_to_59999_dollars', 
    'household_income_from_60000_to_99999_dollars', 
    'household_income_from_100000_to_149999_dollars', 
    'household_income_from_150000_to_199999_dollars', 
    'household_income_more_than_200000_dollars', 
    'annual_individual_earnings_less_than_10000_dollars', 
    'annual_individual_earnings_from_10000_to_19999_dollars', 
    'annual_individual_earnings_from_20000_to_29999_dollars', 
    'annual_individual_earnings_from_30000_to_39999_dollars', 
    'annual_individual_earnings_from_40000_to_49999_dollars', 
    'annual_individual_earnings_from_50000_to_64999_dollars', 
    'annual_individual_earnings_from_65000_to_74999_dollars', 
    'annual_individual_earnings_from_75000_to_99999_dollars', 
    'annual_individual_earnings_more_than_100000_dollars', 
    'earnings_source_fulltime_with_earnings', 
    'earnings_source_parttime_with_earnings', 
    'earnings_source_no_earnings', 'Sunrise_Sunset', 'Civil_Twilight', 
    'Nautical_Twilight', 'Astronomical_Twilight'
]

# Drop the specified columns from the DataFrame
filtered_df = merged_df.drop(columns=columns_to_exclude)

# Print the shape of the filtered DataFrame
print(filtered_df.shape)


In [ ]:
# Set pandas display option to show all columns
pd.set_option('display.max_columns', None)

# Display the first few rows of the filtered DataFrame
filtered_df.head()


In [ ]:
# Group by (City, State)
grouped = filtered_df.groupby(['City', 'State'])

# Calculate total records per group
total_records = grouped.size().rename('Total_Records')

# Filter for groups with more than 50,000 records
filtered_groups = total_records[total_records > 50000].index

# Subset the DataFrame for valid groups only
filtered_df_subset = filtered_df[filtered_df.set_index(['City', 'State']).index.isin(filtered_groups)]

# Recalculate missing values and align with filtered groups
total_missing = (
    filtered_df_subset.groupby(['City', 'State'])
    .apply(lambda group: group.isna().sum().sum())
    .rename('Total_Missing')
)

# Recalculate total records for filtered groups
total_records = total_records.loc[filtered_groups]

# Calculate missing ratio
missing_ratio = (total_missing / total_records).rename('Missing_Ratio')

# Combine results into a single DataFrame
result = pd.concat([total_records, total_missing, missing_ratio], axis=1)

# Sort by missing ratio
result = result.sort_values(by='Missing_Ratio')

# Display the result
result.head()


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(result)

In [ ]:
top_cities_states = result.index[:15]
print(top_cities_states)

In [ ]:
# Filter filtered_df to include only these (City, State) pairs
filtered_df_top = filtered_df[filtered_df.set_index(['City', 'State']).index.isin(top_cities_states)]

In [ ]:
filtered_df_top.shape

In [ ]:
######################### H3 ID ############################
import h3

# Function to create H3 IDs based on latitude, longitude, and resolution level
def create_h3_ids(row, level):
    lat, lng = row['Start_Lat'], row['Start_Lng']
    h3_id = h3.geo_to_h3(lat, lng, level)
    return h3_id

# Create the new DataFrame with the required columns for H3 resolution level 7
new_df_7 = pd.DataFrame()
new_df_7['H3 ID Level 7'] = filtered_df_top.apply(lambda row: create_h3_ids(row, 7), axis=1)  # Generate H3 IDs

# Remove duplicate H3 ID Level 7 entries
new_df_7 = new_df_7.drop_duplicates(subset=['H3 ID Level 7'])

# Reset the index to create unique row numbers for Area ID 7
new_df_7.reset_index(drop=True, inplace=True)
new_df_7.reset_index(inplace=True)

# Rename the index column to 'Area ID 7'
new_df_7.rename(columns={'index': 'Area ID 7'}, inplace=True)

# Display the first few rows of the resulting DataFrame
new_df_7.head()


In [ ]:
new_df_7.shape

In [ ]:
new_df_7.head()

In [ ]:
print("********************* Total Zipcodes present in the top 15 cities: " + str(len(set(filtered_df_top['Zipcode']))))

In [ ]:
filtered_df_top.shape

In [ ]:
filtered_df_top.head()

In [ ]:
# Add the 'H3 ID Level 7' and the 'Area ID 7' for each record (based on the latitude and longitude)
filtered_df_top['H3 ID Level 7'] = filtered_df_top.apply(lambda row: create_h3_ids(row, 7), axis=1)
filtered_df_top = filtered_df_top.merge(new_df_7[['H3 ID Level 7', 'Area ID 7']], on='H3 ID Level 7', how='left')

In [ ]:
filtered_df_top.shape

In [ ]:
filtered_df_top.head()

In [ ]:
# Count records with no missing values per 'Area ID 7'
non_missing_counts = (
    filtered_df_top.dropna()
    .groupby('Area ID 7')
    .size()
    .reset_index(name='Non_Missing_Records')
)

# Count total records per 'Area ID 7'
total_counts = (
    filtered_df_top.groupby('Area ID 7')
    .size()
    .reset_index(name='Total_Records')
)

# Merge the two results to get both counts side-by-side
area_counts = pd.merge(non_missing_counts, total_counts, on='Area ID 7', how='right')

# Fill missing Non_Missing_Records with 0 (if an Area ID has no completely non-missing rows)
area_counts['Non_Missing_Records'] = area_counts['Non_Missing_Records'].fillna(0).astype(int)

# Add City and State information (if applicable to Area ID 7)
city_state_mapping = filtered_df_top[['Area ID 7', 'City', 'State']].drop_duplicates()
area_counts = pd.merge(area_counts, city_state_mapping, on='Area ID 7', how='left')

# Reorder columns for better readability
area_counts = area_counts[['Area ID 7', 'City', 'State', 'Non_Missing_Records', 'Total_Records']]

# Display the final result
print(area_counts)


In [ ]:
area_counts_under50 = area_counts[area_counts['Total_Records'] >= 100]
area_counts_under50.shape

In [ ]:
area_counts_under50.head()

In [ ]:
city_counts_before = area_counts.groupby('City').size().reset_index(name='Total_AreIDs_before_deletion')
print(city_counts_before)
city_counts = area_counts_under50.groupby('City').size().reset_index(name='Total_AreIDs_after_deletion')
print(city_counts)

In [ ]:
area_counts_under50['Non_Missing_Ratio'] = area_counts_under50['Non_Missing_Records'] / area_counts_under50['Total_Records']
sorted_area_counts = area_counts_under50.sort_values(by=['City', 'Non_Missing_Ratio'], ascending=[True, False])
print(sorted_area_counts)

In [ ]:
filtered_data = sorted_area_counts[sorted_area_counts['Non_Missing_Ratio'] >= 0.95]
city_counts = filtered_data.groupby('City').size().reset_index(name='Total_AreIDs_after_deleting_below_95')
print(city_counts)

In [ ]:
filtered_data.shape

In [ ]:
filtered_data.head()

In [ ]:
# Keep only the Rows from filtered_df_top whose Area ID is present in filtered_data
area_id_7_list = filtered_data['Area ID 7'].unique()
filtered_df_top = filtered_df_top[filtered_df_top['Area ID 7'].isin(area_id_7_list)]

In [ ]:
filtered_df_top.shape

In [ ]:
filtered_df_top.head()

In [ ]:
import pandas as pd

# Define the time range
#start_time = pd.Timestamp('2016-03-22 00:00:00')
start_time = pd.Timestamp('2016-06-01 00:00:00')  # Adjust start time as needed
end_time = pd.Timestamp('2023-03-31 23:59:59')
time_bins = pd.date_range(start=start_time, end=end_time, freq='3H')

# Ensure 'Start_Time' is a datetime type
filtered_df_top['Start_Time'] = pd.to_datetime(filtered_df_top['Start_Time'])

# Pre-bin the 'Start_Time' into intervals
filtered_df_top['time_bin'] = filtered_df_top['Start_Time'].dt.floor('3H')
# Optional: Calculate time_bin_end if needed for further analysis
# filtered_df_top['time_bin_end'] = filtered_df_top['time_bin'] + pd.Timedelta(hours=12)

# Group by 'Area ID 7' and 'time_bin' to calculate the required aggregates
grouped = filtered_df_top.groupby(['Area ID 7', 'time_bin']).agg(
    Total_Accidents=('Severity', 'size'),
    Acc_Severity=('Severity', lambda x: list(x.unique()))
).reset_index()

# Display the grouped data
print(grouped)


In [ ]:
# Create all combinations of 'Area ID 7' and 'time_bins'
all_combinations = pd.MultiIndex.from_product(
    [filtered_df_top['Area ID 7'].unique(), time_bins], names=['Area ID 7', 'time_bin']
)
time_series_df = pd.DataFrame(index=all_combinations).reset_index()

# Merge the grouped results back onto the time_series_df
time_series_df = time_series_df.merge(grouped, on=['Area ID 7', 'time_bin'], how='left')

# Display the time series DataFrame
print(time_series_df)

In [ ]:
# Fill NaN values for missing time bins
time_series_df['Total_Accidents'] = time_series_df['Total_Accidents'].fillna(0).astype(int)

In [ ]:
# Replace NaN in 'Acc_Severity' with [0] for proper handling
time_series_df['Acc_Severity'] = time_series_df['Acc_Severity'].apply(
    lambda x: [0] if isinstance(x, float) and pd.isna(x) else x
)


In [ ]:
time_series_df.head()

In [ ]:
for column in filtered_df_top.columns:
    if column not in ['Start_Time', 'Severity', 'Area ID 7', 'time_bin']:
        time_series_df[column] = time_series_df['Area ID 7'].map(
            filtered_df_top.groupby('Area ID 7')[column].first()
        )

In [ ]:
time_series_df.head()

In [ ]:
time_series_df.to_csv('/fs/ess/PAS0536/ppgneogi/TrafficEvents/TimeSeries_WithoutWeather.csv', index=False)

In [ ]:
#import pandas as pd
#time_series_df = pd.read_csv('TimeSeries_WithoutWeather2.csv')

In [ ]:
#!pip install openpyxl

In [ ]:
import pandas as pd
import os

# Set the file path and column selection
file_path = ""  # Update with the correct path
columns_to_select = [
    'valid', 'tmpf', 'dwpf', 'relh', 'drct', 'sknt', 'p01i', 
    'alti', 'vsby', 'skyc1'
]
start_time = pd.Timestamp('2016-06-01 00:00:00')
end_time = pd.Timestamp('2023-03-31 23:59:59')

# List of files to process
files = [
    "SanDiegoWeather_imputed", "SacramentoWeather_imputed", "LosAngelesWeather_imputed", 
    "BatonRougeWeather_imputed", "OrlandoWeather_imputed", "MiamiWeather_imputed", 
    "NashvilleWeather_imputed", "MinneapolisWeather_imputed", "CharlotteWeather_imputed", 
    "RaleighWeather_imputed", "PhoenixWeather_imputed", "HoustonWeather_imputed", 
    "DallasWeather_imputed", "AustinWeather_imputed", "AtlantaWeather_imputed"
]

# Dictionary to hold the dataframes
weather_dataframes = {}

for file_name in files:
    # Load the data
    file = os.path.join(file_path, file_name + ".xlsx")
    df = pd.read_excel(file)

    # Select relevant columns
    df = df[columns_to_select]

    # Convert 'valid' column to datetime
    df['valid'] = pd.to_datetime(df['valid'])

    # Filter data within the specified time range
    df = df[(df['valid'] >= start_time) & (df['valid'] <= end_time)]

    # Create 3-hour time bins
    df['time_bin'] = (df['valid'] - start_time).dt.total_seconds() // (3 * 3600)
    df['time_bin'] = df['time_bin'].astype(int)

    # Group by time_bin and aggregate
    aggregated_df = df.groupby('time_bin').agg(
        tmpf=('tmpf', 'mean'),
        dwpf=('dwpf', 'mean'),
        relh=('relh', 'mean'),
        drct=('drct', 'mean'),
        sknt=('sknt', 'mean'),
        p01i=('p01i', 'mean'),
        alti=('alti', 'mean'),
        vsby=('vsby', 'mean'),
        skyc1=('skyc1', 'last')  # Take the last value for 'skyc1'
    ).reset_index()

    # Add the start time of the time bin
    aggregated_df['time_bin_start'] = aggregated_df['time_bin'].apply(
        lambda x: start_time + pd.Timedelta(hours=3 * x)
    )

    # Drop the time_bin column and reorder
    aggregated_df = aggregated_df.drop(columns=['time_bin'])
    aggregated_df = aggregated_df[['time_bin_start'] + 
                                   [col for col in aggregated_df.columns if col != 'time_bin_start']]

    # Save the dataframe in the dictionary
    weather_dataframes[file_name] = aggregated_df

In [ ]:
weather_dataframes.keys()

In [ ]:
# Mapping the old keys to the proper city names
city_name_mapping = {
    "SanDiegoWeather_imputed": "San Diego",
    "SacramentoWeather_imputed": "Sacramento",
    "LosAngelesWeather_imputed": "Los Angeles",
    "BatonRougeWeather_imputed": "Baton Rouge",
    "OrlandoWeather_imputed": "Orlando",
    "MiamiWeather_imputed": "Miami",
    "NashvilleWeather_imputed": "Nashville",
    "MinneapolisWeather_imputed": "Minneapolis",
    "CharlotteWeather_imputed": "Charlotte",
    "RaleighWeather_imputed": "Raleigh",
    "PhoenixWeather_imputed": "Phoenix",
    "HoustonWeather_imputed": "Houston",
    "DallasWeather_imputed": "Dallas",
    "AustinWeather_imputed": "Austin",
    "AtlantaWeather_imputed": "Atlanta"
}

# Renaming the keys in the dictionary
weather_dataframes = {
    city_name_mapping[key]: value for key, value in weather_dataframes.items()
}

# Display the updated keys
print(weather_dataframes.keys())


In [ ]:
weather_dataframes['San Diego'].columns

In [ ]:
weather_dataframes['San Diego'].head()

In [ ]:
for city, df in weather_dataframes.items():
    # Convert 'time_bin_start' to string and rename it to 'time_bin'
    df.rename(columns={'time_bin_start': 'time_bin'}, inplace=True)
    df['time_bin'] = df['time_bin'].astype(str)
    df['City'] = city  # Add City column
    # Update the dictionary with the modified DataFrame
    weather_dataframes[city] = df


In [ ]:
weather_dataframes['San Diego'].head()

In [ ]:
weather_dataframes['San Diego']['time_bin'].dtype

In [ ]:
combined_weather_df = pd.concat(weather_dataframes.values(), ignore_index=True)

In [ ]:
combined_weather_df.head()

In [ ]:
time_series_df['time_bin'] = time_series_df['time_bin'].astype(str)

In [ ]:
time_series_df['time_bin'].dtype

In [ ]:
time_series_df.head()

In [ ]:
final_merged_df = pd.merge(time_series_df, combined_weather_df, on=['City', 'time_bin'], how='left')

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
final_merged_df.head(10)